# Spectral Loss for Fundamental Mode of N2 as outer air region narrows

Here we compare the spectral loss for the N2 configuration to that of N1 as we decrease the outer air region of N2. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from scipy.optimize import newton

First we load and visualize the spectral loss for N1

# Initialize

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
outer = 'h2'
nu = 1

N1betas = np.load('data/N1_wls.npy')

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass]

d = 4.0775e-05
ts = [d , 1e-5, d, 1.5e-5]

A = BraggExact(ts=ts, ns=ns, no_mesh=True)

In [ ]:
# Formula for loss spikes from article

ms = np.arange(11,15,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d = 15*2/3*1e-6

ls = (2 * n1 * d / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver
ls

In [ ]:
fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nN1 Spectral Loss Profile\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N1betas.imag, color='green', linewidth=1.2)
ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')

# Base N2 Study

In [ ]:
betas1= np.zeros_like(wls, dtype=complex)

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, maxhs=[100 ,100, 100, 100], wl=wl, ns=ns)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas1[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -betas1.imag, color='green', linewidth=1.2, label='base_N2')
ax.plot(wls, -N1betas.imag, color='orange', linewidth=.9, label='N1')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Make air region larger

This is without filtering out the higher mode though, which I need to do, but this gets it start being set up.

In [ ]:
N2betas_t1 = np.zeros_like(wls, dtype=complex)

t1 = 1.5
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t1[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N1betas.imag, color='orange', linewidth=.9, label='N1')
ax.plot(wls, -betas1.imag, color='green', linewidth=1.2, label='N2 t=1')
ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Make it smaller


In [ ]:
N2betas_t0 = np.zeros_like(wls, dtype=complex)

t1 = .5
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t0[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N1betas.imag, color='orange', linewidth=.9, label='N1')
ax.plot(wls, -N2betas_t0.imag, color='red', linewidth=1.2, label='N2 t=.5')
ax.plot(wls, -betas1.imag, color='green', linewidth=1.2, label='N2 t=1')
# ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Even smaller

# Make it smaller


In [ ]:
N2betas_t00 = np.zeros_like(wls, dtype=complex)

t1 = .25
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t00[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\nDecreasing thickness of Outer Air Region\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N1betas.imag,  linewidth=.9, label='N1')
ax.plot(wls, -betas1.imag,linewidth=2, label='N2 t=1')
ax.plot(wls, -N2betas_t0.imag, linewidth=1.5, label='N2 t=.5')
ax.plot(wls, -N2betas_t00.imag, linewidth=1, label='N2 t=.25')

# ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Tiny?

In [ ]:
N2betas_t000 = np.zeros_like(wls, dtype=complex)

t1 = .05
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t000[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\nDecreasing thickness of Outer Air Region\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N1betas.imag,  linewidth=.9, label='N1')
ax.plot(wls, -betas1.imag,linewidth=2, label='N2 t=1')
ax.plot(wls, -N2betas_t0.imag, linewidth=1.5, label='N2 t=.5')
ax.plot(wls, -N2betas_t00.imag, linewidth=1, label='N2 t=.25')
ax.plot(wls, -N2betas_t000.imag, linewidth=1, label='N2 t=.05')

# ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Ridiculous?

In [ ]:
N2betas_t0000 = np.zeros_like(wls, dtype=complex)

t1 = .005
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t0000[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\nDecreasing thickness of Outer Air Region\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N1betas.imag,  linewidth=.9, label='N1')
ax.plot(wls, -betas1.imag,linewidth=2, label='N2 t=1')
ax.plot(wls, -N2betas_t0.imag, linewidth=1.5, label='N2 t=.5')
ax.plot(wls, -N2betas_t00.imag, linewidth=1, label='N2 t=.25')
ax.plot(wls, -N2betas_t000.imag, linewidth=1, label='N2 t=.05')
ax.plot(wls, -N2betas_t0000.imag, linewidth=1, label='N2 t=.005')

# ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Impossible

In [ ]:
N2betas_t00000 = np.zeros_like(wls, dtype=complex)

t1 = .0005
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t00000[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\nDecreasing thickness of Outer Air Region\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N1betas.imag,  linewidth=.9, label='N1')
ax.plot(wls, -betas1.imag,linewidth=2, label='N2 t=1')
ax.plot(wls, -N2betas_t0.imag, linewidth=1.5, label='N2 t=.5')
ax.plot(wls, -N2betas_t00.imag, linewidth=1, label='N2 t=.25')
ax.plot(wls, -N2betas_t000.imag, linewidth=1, label='N2 t=.05')
ax.plot(wls, -N2betas_t0000.imag, linewidth=1, label='N2 t=.005')
ax.plot(wls, -N2betas_t00000.imag, linewidth=1, label='N2 t=.0005')

# ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


In [ ]:
N0_betas = np.load('data/N0_betas.npy')

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\nDecreasing thickness of Outer Air Region\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

# ax.plot(wls, -N1betas.imag,  linewidth=.9, label='N1')
ax.plot(wls, -betas1.imag,linewidth=2, label='N2 t=1')
ax.plot(wls, -N2betas_t0.imag, linewidth=1.5, label='N2 t=.5')
ax.plot(wls, -N2betas_t00.imag, linewidth=1, label='N2 t=.25')
ax.plot(wls, -N2betas_t000.imag, linewidth=1, label='N2 t=.05')
ax.plot(wls, -N2betas_t0000.imag, linewidth=1, label='N2 t=.005')
ax.plot(wls, -N2betas_t00000.imag, linewidth=1, label='N2 t=.0005')
ax.plot(wls, -N0_betas.imag,  linewidth=1, linestyle='--', label='N0')

# ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')

#### Takeaways from the above figure:

- Our original thickness of outer air was too much: we decreased our loss going to half the thickness.

- After that, the loss increases again (at least at values of t studied here) and appear to converge to the loss spectra of the N0 configuration.

- At low outer air thickness, the spikes smooth out.  Would be interesting to see what modes look like there.

- Predicted loss peaks shift as t decreases.

- Would be interesting to know if there is a predicted magnitude of loss at spikes.  Formula just gives locations, but are these supposed to give infinite loss?